In [81]:
import numpy as np
import pandas as pd
from collections import Counter
from math import log


In [82]:
dataframe=pd.DataFrame([[14000, 20250, 2, 1],
                         [15000, 20000, 3, 0],
                         [10000, 10250, 2, 1],
                         [20000, 20250, 3, 0],
                         [22000, 20000, 4, 1],
                         [12000, 10250, 2, 1],
                         [19800, 30250, 3, 0],
                         [13000, 14000, 2, 0],
                         [14000, 30250, 3, 0],
                         [19000, 23250, 2, 1],
                         [17000, 65000, 4, 1],
                         [9000, 30250, 2, 0]],
                      columns=['montant','code_postale','nb_operations','fraude'])



In [83]:
dataframe

,montant,code_postale,nb_operations,fraude
0,14000,20250,2,1
1,15000,20000,3,0
2,10000,10250,2,1
3,20000,20250,3,0
4,22000,20000,4,1
5,12000,10250,2,1
6,19800,30250,3,0
7,13000,14000,2,0
8,14000,30250,3,0
9,19000,23250,2,1


In [84]:
class decision_tree_regressor:
    #cette classe contient toutes les fonctions'''
    def __init__(self,target,dataframe):
        #cette fonction est pout initialiser les variables
        #target:la variable cible à classifier
        #dataframe:les données d'appraentissage
        self.target=target
        self.dataframe=dataframe
        
    def quantitative_split(self,feature,value,dataset):
        #cette fonction divise les données en focntion de valeur value de la variable quatitative feature
        ''''
        INPUT:
            feature:integer correspond à la variable à séparer
            value:integer correspond à la valeur à laquelle séparer
        OUTPUT:
            left:dataframe avec les données ou featue est plus petie ou égale à value
            right:dataframe avec les données ou featue est plus grande à value
        '''
        left=dataset[dataset.loc[:,feature]<=value]
        right=dataset[dataset.loc[:,feature]>value]
        
        return left,right
    
    def qualitative_split(self,feature,value,dataset):
        #cette fonction divise les données en focntion de valeur 'value' de la variable qualitative 'feature'
        '''
        INPUT:
            feature:integer correspond à la variable à séparer
            value:integer correspond à la valeur à laquelle séparer
        OUTPUT:
            left:dataframe avec les données ou featue est égale à value
            right:dataframe avec les données ou featue est différentde de value
        '''
        left=dataset[dataset.loc[:,feature]==value]
        right=dataset[dataset.loc[:,feature]!=value]
        
        return left,right
    
    def entropy(self,dataset):
        #cette fonction calcule l'entropy 
        rows=dataset[self.target]
        #calcule le nombre de valeur distincts
        counts = Counter(rows)
        # Calcule l'entropy 
        entropy = 0
        for count in counts.values():
            p = count / len(dataset)
            entropy -= p * (log(p)/log(2))
        return entropy
   

    def split_evaluator(self,left_dataset,right_dataset):
        #calculer le cout de séparation de noeud en deux branches
        '''
        INPUT:
            left_dataset:dataset de la branche de gauche
            right_dataset:dataset de la branche de droite
        OUTPUT:
            cost:cout de la séparation
        '''
        left_eval=self.entropy(left_dataset)
        nb_left=left_dataset.shape[0]
        right_eval=self.entropy(right_dataset)
        nb_right=right_dataset.shape[0]
        nb_total=nb_left+nb_right
        cost=nb_left/nb_total * left_eval + nb_right/nb_total * right_eval
        
        return cost
    
    def test_qualitative(self, dataset, feature):
        #tester toutes les séparations possibles d'une variable qualitative
        '''
        INPUT 
           - dataset : dataset à évaluer
           - feature : variable du dataset à évaluer
        OUTPUT
           - df_eval : dataframe contenant le coût de chaque séparation
        ''' 
        df_eval = pd.DataFrame([], columns=('feature', 'value', 'nature', 'cost'))
        for value in dataset.loc[:, feature].unique() :
            left, right = self.qualitative_split(feature, value, dataset)
            cost_result = self.split_evaluator(left, right)
            df_eval = pd.concat([df_eval,pd.DataFrame([[feature, value,'qualitative',cost_result]],columns=('feature', 'value', 'nature', 'cost'))])
            
        return df_eval
      
        
    def test_quantitative(self, dataset, feature):
        #Tester toutes les séparations possibles d'une variable quantitative
        '''
        INPUT 
            - dataset : dataset à évaluer
            - feature : variable du dataset à évaluer
        OUTPUT 
           - df_eval : dataframe contenant le coût de chaque séparation
        '''
        df_eval = pd.DataFrame([], columns=('feature', 'value', 'nature', 'cost'))
        for value in dataset.loc[:, feature].unique()  : 
            left, right = self.quantitative_split(feature, value, dataset)
            cost_result = self.split_evaluator(left, right)
            df_eval = pd.concat([df_eval,pd.DataFrame([[feature, value,'quantitative',cost_result]],columns=('feature', 'value', 'nature', 'cost'))])
        return df_eval
    
    
    def find_best_split(self, dataset):
        #Trouver la meilleure séparation
        '''
        INPUT 
           - dataset : jeu de données à séparer
        OUTPUT 
           - def_eval : dataset contenant: 
               'feature' variable à séparer
               'value' la valeur à laquelle séparer la variable
               'nature' la nature de la variable 
               'cost' le coût de cette séparation
        '''
        df_eval = pd.DataFrame([], columns=('feature', 'value', 'nature', 'cost'))
        columns = dataset.columns[np.logical_not(dataset.columns == self.target)]
        for column in columns : 
            if len(dataset[column].unique()) >= 3 :
                df_eval = pd.concat([df_eval,self.test_quantitative(dataset, column)])
            elif len(dataset[column].unique()) == 2 :
                df_eval = pd.concat([df_eval,self.test_qualitative(dataset, column)])

        df_eval = df_eval.reset_index(drop=True)
        
        #index du feature qui le cout minimum
        idx_cost_min = df_eval['cost'].idxmin(axis=0, skipna=True)

        return df_eval.iloc[idx_cost_min, :]
    
    
 
    def create_feuille(self, dataset):
        #Création d'une feuille 
        '''
        INPUT 
           - dataset : dataset de la feuille à construire
        OUTPUT 
           - feuille : la classe feuille créée avec les informations de notre dataset
        ''' 
        labels = dataset[self.target]        
        prediction = np.mean(labels)

        return feuille(dataset, prediction)
    
    def training(self, dataset):
        #Cette fonction va construire l'arbre de décision en fonction des paramètres fournir à l'initialisation de cette classe.
        
      
        # Si le dataset est pure nous créons une feuille
        if len(dataset[self.target].unique())==1 :
            return self.create_feuille(dataset)

        # Recherche de la meilleur séparation
        split_eval = self.find_best_split(dataset)
        # Si le coût obtenu après séparation est moins bon que le coût actuel,création d'une feuille avec le dataset actuel
        if split_eval['cost'] >= self.entropy(dataset) :
            return self.create_feuille(dataset)
      

        # Séparation du dataset selon la nature de la variable choisie
        if split_eval['nature'] == 'qualitative' :
            left_branch, right_branch = self.qualitative_split(split_eval['feature'], split_eval['value'], dataset)
        elif split_eval['nature'] == 'quantitative' :
            left_branch, right_branch = self.quantitative_split(split_eval['feature'], split_eval['value'], dataset)

        # Entraînement récursif de la branche de gauche
        left_node = self.training(left_branch)

        # Entraînement récrusif de la branche de droite
        right_node = self.training(right_branch)
        
        
        # On retourne la racine de l'arbre
        return neoud(split_eval['feature'], 
                  split_eval['value'], 
                  split_eval['cost'], 
                  split_eval['nature'],
                  left_node,
                  right_node)
    
    
        


In [85]:
class neoud:
    #Cette classe a pour but de représenter les branches de notre arbre de regression
    def __init__(self, feature, value, cost, nature, left_branch, right_branch):
        self.feature = feature
        self.value = value
        self.nature = nature
        self.left_branch = left_branch
        self.right_branch = right_branch

    def __split__(self):
        if self.nature == 'quantitative' :
            return self.feature + ' <= ' + str(self.value)
        elif self.nature == 'qualitative' :
            return self.feature + ' == ' + str(self.value)

In [86]:
def print_tree(neoud, spacing=""):
    #Affichage de l'arbre de décision
    '''
    INPUT 
     - node : branche à afficher
     - spacings : espace à afficher en fonction de la profondeur de la branche
    '''

    # Différents affichages si c'est une feuille 
    #vérifier si le neoud est une classe finale=feuille
    if isinstance(neoud, feuille):
        print (spacing + "Prediction", neoud.prediction)
        return

    # Affichage de la condition de la séparation
    print (spacing + neoud.__split__())

    # Dans le cas où la condition est vérifiée
    print (spacing + '--> True:')
    print_tree(neoud.left_branch, spacing + "  ")

    # Dans le cas où la condition n'est pas vérifiée
    print (spacing + '--> False:')
    print_tree(neoud.right_branch, spacing + "  ")

In [87]:
class feuille:
    #Cette classe a pour but de représenter les feuilles de l'arbre
    def __init__(self, dataset, prediction):
        self.dataset = dataset
        self.prediction = prediction
        
        

In [88]:
tree = decision_tree_regressor('fraude', dataframe)
tree_trained = tree.training(dataframe)
print_tree(tree_trained)



code_postale <= 10250
--> True:
  Prediction 1.0
--> False:
  nb_operations <= 3
  --> True:
    nb_operations == 2
    --> True:
      montant <= 13000
      --> True:
        Prediction 0.0
      --> False:
        Prediction 1.0
    --> False:
      Prediction 0.0
  --> False:
    Prediction 1.0
